In [1]:
# Import libraries
import os # file system operations

from dotenv import load_dotenv # loading env variables from .env file
from openai import OpenAI # openai api for translation and image generation
from pydantic import BaseModel #create structured output from API

from PIL import Image # image processing operations
import requests # http web operations
from io import BytesIO # image manipulation, binary data handling

from tqdm import tqdm # monitor progress of model operations
import json # convert json string to python dictionary
import time # to add delay to work around API rate limit error

In [2]:
# Load environment variable (OpenAI API key) from .env file
load_dotenv()

# Set up OpenAI client
client = OpenAI()

In [3]:
# Define simple topic-hindiwords dictionary
topics = {
    "Animals": ["बिल्ली", "कुत्ता", "हाथी", "जिराफ", "खरगोश"],
    "Fruits": ["सेब", "केला", "चेरी", "अंगूर", "संतरा"],
    "Vehicles": ["कार", "बाइक", "विमान", "जहाज","बस"],
    "Things": ["किताब", "कुर्सी", "टेबल", "पेंसिल", "घड़ी"],
    "Places": ["स्कूल", "घर", "पार्क", "हॉस्पिटल", "पुस्तकालय"]
}

Step 1 - Translation:
1. Input: Hindi word values from `topics` dictionary
2. Output: English translated words using [OpenAI Chat Completions API](https://platform.openai.com/docs/guides/chat-completions)

    _Model used_: `gpt-4o-mini` as it is light weight, lesser cost and faster, for simpler use case. 

In [4]:
# Defining structured output from API with Pydantic object
class HindiResponse(BaseModel):
    english_text: list[str]

# Function to use OpenAI Chat Completions endpoint for translation based on prompt
def translate_text(text, topic):
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"Translate the list of user words from Hindi to English. Once translated, ensure the words are returned in the same order as user input. The words in the list belongs to the topic: {topic}"},
            {"role": "user", "content": f"{text}"}
        ], response_format=HindiResponse 
    )
    return json.loads(response.choices[0].message.content)['english_text']

In [5]:
# Call Chat Completions function for Translation from Hindi to English
translated_topics = {}

for topic, words_list in tqdm(topics.items()):
    translated_topics[topic] = translate_text(words_list, topic)
    translated_topics[topic] = [word.title() for word in translated_topics[topic]]

translated_topics

100%|██████████| 5/5 [00:18<00:00,  3.73s/it]


{'Animals': ['Cat', 'Dog', 'Elephant', 'Giraffe', 'Rabbit'],
 'Fruits': ['Apple', 'Banana', 'Cherry', 'Grapes', 'Orange'],
 'Vehicles': ['Car', 'Bike', 'Airplane', 'Ship', 'Bus'],
 'Things': ['Book', 'Chair', 'Table', 'Pencil', 'Watch'],
 'Places': ['School', 'House', 'Park', 'Hospital', 'Library']}

Step 2 - Image Generation:
1. Input: English translated word values from `translated_topics` dictionary as text prompts
2. Output: Images created from text prompt using [OpenAI Image Generation API](https://platform.openai.com/docs/guides/images/usage)

    _Model used_: `dall-e-2` (default option) as it allows returning smallest size image of dimension 256x256 (smallest image size for `dall-e-3` based on OpenAI documentation is 1024x1024) and has flexibility to return multiple images per prompt, allowing review and selection of one best option for downstream app consumption. 

In [6]:
# Function to generate and save images based on a dictionary of topics and words
def generate_and_save_images(topics_dict, base_dir="./generated_images"):
    # Create the base directory to store all generated images
    os.makedirs(base_dir, exist_ok=True)

    # Iterate over each topic and the corresponding list of words
    for topic, words_list in tqdm(translated_topics.items()):
        # Create a subfolder for each topic
        topic_dir = os.path.join(base_dir, topic)
        os.makedirs(topic_dir, exist_ok=True)

        # Iterate over each word in the list
        for word in words_list:
            # Generate images for the current word and topic using OpenAI Image Generation API
            response = client.images.generate(
                prompt=f"Create a fun cartoon image of a {word} that belongs to the topic {topic}. The image should be safe for consumption by children less than 10 years for educational purposes. There should be no letters, numbers or any characters anywhere in the image.",
                size="256x256",
                n=5
            )

            # Save each generated image of each word to the corresponding topic's subfolder
            for i, image_data in enumerate(response.data):
                image_url = image_data.url  # Access the URL of the current image
                
                # Fetch the image from the URL
                image_response = requests.get(image_url)
                
                # Create an image from the fetched URL content
                image = Image.open(BytesIO(image_response.content))
                
                # Define the filename with the word and index
                image_filename = f"generated_image_{word}_{i+1}.png"
                
                # Full path to save the image
                image_path = os.path.join(topic_dir, image_filename)
                
                # Save the image locally
                image.save(image_path)
                print(f"Saved image {i+1} for word '{word}' in topic '{topic}' to {image_path}")

            # Introduce a delay to stay within the rate limit of 5 images per minute 
            time.sleep(60)  # Wait for 60 seconds (1 minute) before the next request

# Generate and save images for each topic and word in the dictionary
generate_and_save_images(translated_topics)

  0%|          | 0/5 [00:00<?, ?it/s]

Saved image 1 for word 'Cat' in topic 'Animals' to ./generated_images/Animals/generated_image_Cat_1.png
Saved image 2 for word 'Cat' in topic 'Animals' to ./generated_images/Animals/generated_image_Cat_2.png
Saved image 3 for word 'Cat' in topic 'Animals' to ./generated_images/Animals/generated_image_Cat_3.png
Saved image 4 for word 'Cat' in topic 'Animals' to ./generated_images/Animals/generated_image_Cat_4.png
Saved image 5 for word 'Cat' in topic 'Animals' to ./generated_images/Animals/generated_image_Cat_5.png
Saved image 1 for word 'Dog' in topic 'Animals' to ./generated_images/Animals/generated_image_Dog_1.png
Saved image 2 for word 'Dog' in topic 'Animals' to ./generated_images/Animals/generated_image_Dog_2.png
Saved image 3 for word 'Dog' in topic 'Animals' to ./generated_images/Animals/generated_image_Dog_3.png
Saved image 4 for word 'Dog' in topic 'Animals' to ./generated_images/Animals/generated_image_Dog_4.png
Saved image 5 for word 'Dog' in topic 'Animals' to ./generated_i

 20%|██        | 1/5 [06:45<27:02, 405.71s/it]

Saved image 1 for word 'Apple' in topic 'Fruits' to ./generated_images/Fruits/generated_image_Apple_1.png
Saved image 2 for word 'Apple' in topic 'Fruits' to ./generated_images/Fruits/generated_image_Apple_2.png
Saved image 3 for word 'Apple' in topic 'Fruits' to ./generated_images/Fruits/generated_image_Apple_3.png
Saved image 4 for word 'Apple' in topic 'Fruits' to ./generated_images/Fruits/generated_image_Apple_4.png
Saved image 5 for word 'Apple' in topic 'Fruits' to ./generated_images/Fruits/generated_image_Apple_5.png
Saved image 1 for word 'Banana' in topic 'Fruits' to ./generated_images/Fruits/generated_image_Banana_1.png
Saved image 2 for word 'Banana' in topic 'Fruits' to ./generated_images/Fruits/generated_image_Banana_2.png
Saved image 3 for word 'Banana' in topic 'Fruits' to ./generated_images/Fruits/generated_image_Banana_3.png
Saved image 4 for word 'Banana' in topic 'Fruits' to ./generated_images/Fruits/generated_image_Banana_4.png
Saved image 5 for word 'Banana' in top

 40%|████      | 2/5 [13:32<20:19, 406.50s/it]

...

 60%|██████    | 3/5 [20:19<13:32, 406.42s/it]

Saved image 1 for word 'Book' in topic 'Things' to ./generated_images/Things/generated_image_Book_1.png
Saved image 2 for word 'Book' in topic 'Things' to ./generated_images/Things/generated_image_Book_2.png
Saved image 3 for word 'Book' in topic 'Things' to ./generated_images/Things/generated_image_Book_3.png
Saved image 4 for word 'Book' in topic 'Things' to ./generated_images/Things/generated_image_Book_4.png
Saved image 5 for word 'Book' in topic 'Things' to ./generated_images/Things/generated_image_Book_5.png
Saved image 1 for word 'Chair' in topic 'Things' to ./generated_images/Things/generated_image_Chair_1.png
Saved image 2 for word 'Chair' in topic 'Things' to ./generated_images/Things/generated_image_Chair_2.png
Saved image 3 for word 'Chair' in topic 'Things' to ./generated_images/Things/generated_image_Chair_3.png
Saved image 4 for word 'Chair' in topic 'Things' to ./generated_images/Things/generated_image_Chair_4.png
Saved image 5 for word 'Chair' in topic 'Things' to ./ge

 80%|████████  | 4/5 [27:03<06:45, 405.61s/it]

Saved image 1 for word 'School' in topic 'Places' to ./generated_images/Places/generated_image_School_1.png
Saved image 2 for word 'School' in topic 'Places' to ./generated_images/Places/generated_image_School_2.png
Saved image 3 for word 'School' in topic 'Places' to ./generated_images/Places/generated_image_School_3.png
Saved image 4 for word 'School' in topic 'Places' to ./generated_images/Places/generated_image_School_4.png
Saved image 5 for word 'School' in topic 'Places' to ./generated_images/Places/generated_image_School_5.png
Saved image 1 for word 'House' in topic 'Places' to ./generated_images/Places/generated_image_House_1.png
Saved image 2 for word 'House' in topic 'Places' to ./generated_images/Places/generated_image_House_2.png
Saved image 3 for word 'House' in topic 'Places' to ./generated_images/Places/generated_image_House_3.png
Saved image 4 for word 'House' in topic 'Places' to ./generated_images/Places/generated_image_House_4.png
Saved image 5 for word 'House' in to

100%|██████████| 5/5 [33:49<00:00, 405.95s/it]
